In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

asignacion_path = 'bases/asignacion/base_asignacion_20240902_ TRABAJADA _ v2.xlsx'
df_asignacion = pd.read_excel(asignacion_path)

In [2]:
cols_asignacion = ['codigo', 'ID_FLAG', 'cont_18', 'nombre', 'TIPO_CARTERA', 'tipo_fondo', 'clave', 'AGENCIA CORRECTA']
df_asignacion = df_asignacion[cols_asignacion]

df_asignacion['cont_18'] = df_asignacion['cont_18'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
df_asignacion['codigo'] = df_asignacion['codigo'].astype('Int64').astype(str).str.zfill(8)

df_asignacion['ID_FLAG'] = 0
conteo_cc = df_asignacion['codigo'].value_counts()
df_asignacion['ID_FLAG'] = df_asignacion['codigo'].map(conteo_cc)

print(df_asignacion.shape)
df_asignacion.head()

(247387, 8)


,codigo,ID_FLAG,cont_18,nombre,TIPO_CARTERA,tipo_fondo,clave,AGENCIA CORRECTA
0,00002569,1,001101075001274140,GLADYS MERELYN ABANTO GONZALES,UNSECURED,NaN,KSTBC,ASESCOM RJ
1,00014583,2,001101235001222660,ADMINISTRACION DE EMPRESAS LIBRERIA EDITORIAL SAC,UNSECURED,FAE,BC,SIN AGENCIA
2,00014583,2,001101235001218442,ADMINISTRACION DE EMPRESAS LIBRERIA EDITORIAL SAC,UNSECURED,FAE,BC,SIN AGENCIA
3,00016896,1,001101305001000046,PABLO AGARIE MIYASATO,UNSECURED,NaN,KSTBC,ASESCOM RJ
4,00017507,1,001108098100538643,AGENCIA DE ADUANA YVAPORES LABARTHE SA,SECURED,NaN,BC,SIN AGENCIA


In [3]:
df_asignacion.to_excel('bases/asignacion/base_asignacion.xlsx', index=False)